### Import required libraries

In [1]:
import json
import networkx as nx
import openai
from groq import Groq
from neo4j import GraphDatabase
import json
import json
import networkx as nx
import time

### Connection to Neo4j

In [2]:
# Neo4j connection details
uri = "bolt://localhost:7687"  
user = "neo4j"                 
password = "123456789"          

# Initialize the Neo4j driver
driver = GraphDatabase.driver(uri, auth=(user, password))

# Function to run a query and return the results
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

### Get all data from neo4j

In [3]:
# Get all nodes
def get_all_nodes(tx):
    query = """
    MATCH (n)
    RETURN labels(n) AS labels, n
    """
    result = tx.run(query)
    nodes = []
    for record in result:
        labels = record["labels"]
        node = record["n"]
        nodes.append({
            "labels": labels,
            "properties": dict(node)
        })
    return nodes

# Perform query
with driver.session() as session:
    all_nodes = session.read_transaction(get_all_nodes)

# Close connection driver
driver.close()

# Result
for node in all_nodes:
    print(f"Labels: {node['labels']}, Properties: {node['properties']}")

/var/folders/37/xpv32ytx11lbg70cx5xlqxr80000gn/T/ipykernel_70031/2248797696.py:20: DeprecationWarning: read_transaction has been renamed to execute_read
  all_nodes = session.read_transaction(get_all_nodes)


Labels: ['Tournament'], Properties: {'year': 2011, 'name': 'Germany 2011', 'id': '251475', 'shortName': 'germany2011'}
Labels: ['Tournament'], Properties: {'year': 2019, 'name': 'France 2019', 'id': '278513', 'shortName': 'france2019'}
Labels: ['Tournament'], Properties: {'year': 1991, 'name': 'China PR 1991', 'id': '3373', 'shortName': 'chinapr1991'}
Labels: ['Tournament'], Properties: {'year': 2003, 'name': 'USA 2003', 'id': '6929', 'shortName': 'usa2003'}
Labels: ['Tournament'], Properties: {'year': 1995, 'name': 'Sweden 1995', 'id': '4654', 'shortName': 'sweden1995'}
Labels: ['Tournament'], Properties: {'year': 1999, 'name': 'USA 1999', 'id': '4644', 'shortName': 'usa1999'}
Labels: ['Tournament'], Properties: {'year': 2007, 'name': 'China 2007', 'id': '10232', 'shortName': 'china2007'}
Labels: ['Tournament'], Properties: {'year': 2015, 'name': 'Canada 2015', 'id': '266030', 'shortName': 'canada2015'}
Labels: ['Team'], Properties: {'name': 'Equatorial Guinea', 'id': '1888631'}
Label

### Encoding graph

In [6]:
def encode_data(data):
    encoded_str = ""
    for idx, node in enumerate(data):
        labels_str = ', '.join(node['labels'])
        properties_str = ', '.join([f"{key}: {value}" for key, value in node['properties'].items()])
        encoded_str += f"Node {idx} has labels [{labels_str}] and ({properties_str}), "
    return encoded_str[:-2]  # Loại bỏ dấu phẩy cuối cùng

# Encode data and print results
encoded_all_data = encode_data(all_nodes)
print(encoded_all_data)

Node 0 has labels [Tournament] and (year: 2011, name: Germany 2011, id: 251475, shortName: germany2011), Node 1 has labels [Tournament] and (year: 2019, name: France 2019, id: 278513, shortName: france2019), Node 2 has labels [Tournament] and (year: 1991, name: China PR 1991, id: 3373, shortName: chinapr1991), Node 3 has labels [Tournament] and (year: 2003, name: USA 2003, id: 6929, shortName: usa2003), Node 4 has labels [Tournament] and (year: 1995, name: Sweden 1995, id: 4654, shortName: sweden1995), Node 5 has labels [Tournament] and (year: 1999, name: USA 1999, id: 4644, shortName: usa1999), Node 6 has labels [Tournament] and (year: 2007, name: China 2007, id: 10232, shortName: china2007), Node 7 has labels [Tournament] and (year: 2015, name: Canada 2015, id: 266030, shortName: canada2015), Node 8 has labels [Team] and (name: Equatorial Guinea, id: 1888631), Node 9 has labels [Team] and (name: Nigeria, id: 1882893), Node 10 has labels [Team] and (name: Australia, id: 1882891), Node

In [7]:
len(encoded_data)

200908

### Get relationships and properties

In [8]:
def get_nodes_and_relationships(tx):
    query = (
        "MATCH (a)-[r]->(b) "
        "RETURN DISTINCT labels(a) AS StartLabels, type(r) AS RelationshipType, labels(b) AS EndLabels, properties(r) AS RelationshipProperties "
        "ORDER BY StartLabels, RelationshipType, EndLabels"
    )
    result = tx.run(query)
    nodes_and_relationships = []
    for record in result:
        nodes_and_relationships.append({
            "StartLabels": record["StartLabels"],
            "RelationshipType": record["RelationshipType"],
            "EndLabels": record["EndLabels"],
            "RelationshipProperties": dict(record["RelationshipProperties"])  # Convert properties to dictionary
        })
    return nodes_and_relationships


with driver.session() as session:
    nodes_and_relationships = session.read_transaction(get_nodes_and_relationships)

# Close connection to driver
driver.close()

/var/folders/37/xpv32ytx11lbg70cx5xlqxr80000gn/T/ipykernel_70031/3342820478.py:19: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/var/folders/37/xpv32ytx11lbg70cx5xlqxr80000gn/T/ipykernel_70031/3342820478.py:20: DeprecationWarning: read_transaction has been renamed to execute_read
  nodes_and_relationships = session.read_transaction(get_nodes_and_relationships)


In [9]:
nodes_and_relationships

[{'StartLabels': ['Match'],
  'RelationshipType': 'IN_TOURNAMENT',
  'EndLabels': ['Tournament'],
  'RelationshipProperties': {}},
 {'StartLabels': ['Person'],
  'RelationshipType': 'COACH_FOR',
  'EndLabels': ['Squad'],
  'RelationshipProperties': {}},
 {'StartLabels': ['Person'],
  'RelationshipType': 'IN_SQUAD',
  'EndLabels': ['Squad'],
  'RelationshipProperties': {}},
 {'StartLabels': ['Person'],
  'RelationshipType': 'PLAYED_IN',
  'EndLabels': ['Match'],
  'RelationshipProperties': {'type': 'Started'}},
 {'StartLabels': ['Person'],
  'RelationshipType': 'PLAYED_IN',
  'EndLabels': ['Match'],
  'RelationshipProperties': {'minuteOn': "66'", 'type': 'Subbed On'}},
 {'StartLabels': ['Person'],
  'RelationshipType': 'PLAYED_IN',
  'EndLabels': ['Match'],
  'RelationshipProperties': {'minuteOn': "83'", 'type': 'Subbed On'}},
 {'StartLabels': ['Person'],
  'RelationshipType': 'PLAYED_IN',
  'EndLabels': ['Match'],
  'RelationshipProperties': {'minuteOn': "84'", 'type': 'Subbed On'}},
 

### Encoding relationships

In [10]:
def encode_relationship(relationship):
    start_labels = ', '.join(relationship['StartLabels'])
    end_labels = ', '.join(relationship['EndLabels'])
    relationship_type = relationship['RelationshipType']
    properties = relationship['RelationshipProperties']

    properties_str = ', '.join([f"{key}: {value}" for key, value in properties.items()]) if properties else "No properties"

    return f"Node [{start_labels}] connect to node  [{end_labels}] with Relationship Type {relationship_type} and Properties {{{properties_str}}}"

encoded_relationship =  ""
# In ra mỗi mối quan hệ đã được encode
for relationship in nodes_and_relationships:
    encoded_relationship += encode_relationship(relationship) + ". "

In [11]:
encoded_relationship

"Node [Match] connect to node  [Tournament] with Relationship Type IN_TOURNAMENT and Properties {No properties}. Node [Person] connect to node  [Squad] with Relationship Type COACH_FOR and Properties {No properties}. Node [Person] connect to node  [Squad] with Relationship Type IN_SQUAD and Properties {No properties}. Node [Person] connect to node  [Match] with Relationship Type PLAYED_IN and Properties {type: Started}. Node [Person] connect to node  [Match] with Relationship Type PLAYED_IN and Properties {minuteOn: 66', type: Subbed On}. Node [Person] connect to node  [Match] with Relationship Type PLAYED_IN and Properties {minuteOn: 83', type: Subbed On}. Node [Person] connect to node  [Match] with Relationship Type PLAYED_IN and Properties {minuteOn: 84', type: Subbed On}. Node [Person] connect to node  [Match] with Relationship Type PLAYED_IN and Properties {minuteOn: 112', type: Subbed On}. Node [Person] connect to node  [Match] with Relationship Type PLAYED_IN and Properties {min

In [12]:
len(encoded_relationship)

28486

### Read data

In [13]:
import json

# Đường dẫn đến file JSON (sau khi đã xóa ký tự BOM)
file_path = 'data_wc.json'

# Đọc nội dung từ file JSON
with open(file_path, 'r', encoding='utf-8-sig') as file:
    try:
        data = json.load(file)
        # Xử lý dữ liệu JSON ở đây
        for item in data:
            # Truy cập và xử lý các thuộc tính của mỗi đối tượng
            pass
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")


In [14]:
for item in data: 
    print(item)

{'n': {'identity': 45, 'labels': ['Person'], 'properties': {'dob': '1977-09-14', 'name': 'Andreia', 'id': '39'}, 'elementId': '45'}}
{'n': {'identity': 46, 'labels': ['Person'], 'properties': {'dob': '1982-11-01', 'name': 'Elaine', 'id': '201416'}, 'elementId': '46'}}
{'n': {'identity': 47, 'labels': ['Person'], 'properties': {'dob': '1982-07-06', 'name': 'Aline', 'id': '212298'}, 'elementId': '47'}}
{'n': {'identity': 48, 'labels': ['Person'], 'properties': {'dob': '1974-10-03', 'name': 'Tania', 'id': '36'}, 'elementId': '48'}}
{'n': {'identity': 49, 'labels': ['Person'], 'properties': {'dob': '1986-07-08', 'name': 'Renata Costa', 'id': '190354'}, 'elementId': '49'}}
{'n': {'identity': 50, 'labels': ['Person'], 'properties': {'dob': '1982-07-07', 'name': 'Rosana', 'id': '178667'}, 'elementId': '50'}}
{'n': {'identity': 51, 'labels': ['Person'], 'properties': {'dob': '1984-01-12', 'name': 'Daniela', 'id': '178665'}, 'elementId': '51'}}
{'n': {'identity': 52, 'labels': ['Person'], 'prop

### Encoding node

In [15]:
# Create a graph using NetworkX for nodes only
graph = nx.Graph()

# Adding nodes into graph
for item in data:
    node_id = item['n']['properties']['id']
    node_labels = item['n']['labels']
    node_properties = item['n']['properties']
    graph.add_node(node_id, labels=node_labels, **node_properties)

# Encoding nodes function with identity
def encode_nodes(graph):
    node_descriptions = []
    for node, props in graph.nodes(data=True):
        labels = props.pop('labels', [])
        labels_str = ', '.join(labels)
        prop_desc = ', '.join([f"{key}: {value}" for key, value in props.items()])
        node_desc = f"Node {node} with label {labels_str} has properties ({prop_desc})"
        node_descriptions.append(node_desc)
    return node_descriptions

# Encode the nodes
encoded_nodes = encode_nodes(graph)

# Format the output
if encoded_nodes:
    encoded_graph_str = '; '.join(encoded_nodes)
    output = f"Encoded graph:\n {encoded_graph_str}."
else:
    output = "No nodes found in the graph."

In [16]:
print(output)

Encoded graph:
 Node 39 with label Person has properties (dob: 1977-09-14, name: Andreia, id: 39); Node 201416 with label Person has properties (dob: 1982-11-01, name: Elaine, id: 201416); Node 212298 with label Person has properties (dob: 1982-07-06, name: Aline, id: 212298); Node 36 with label Person has properties (dob: 1974-10-03, name: Tania, id: 36); Node 190354 with label Person has properties (dob: 1986-07-08, name: Renata Costa, id: 190354); Node 178667 with label Person has properties (dob: 1982-07-07, name: Rosana, id: 178667); Node 178665 with label Person has properties (dob: 1984-01-12, name: Daniela, id: 178665); Node 31 with label Person has properties (dob: 1978-03-03, name: Formiga, id: 31); Node 29 with label Person has properties (dob: 1977-04-30, name: Maycon, id: 29); Node 190358 with label Person has properties (dob: 1986-02-19, name: Marta, id: 190358); Node 190362 with label Person has properties (dob: 1985-05-15, name: Cristiane, id: 190362); Node 251049 with 

### Preparing data

In [17]:
encoded_data = output +  "\nIn this graph: \n" + encoded_relationship 

In [18]:
encoded_data

"Encoded graph:\n Node 39 with label Person has properties (dob: 1977-09-14, name: Andreia, id: 39); Node 201416 with label Person has properties (dob: 1982-11-01, name: Elaine, id: 201416); Node 212298 with label Person has properties (dob: 1982-07-06, name: Aline, id: 212298); Node 36 with label Person has properties (dob: 1974-10-03, name: Tania, id: 36); Node 190354 with label Person has properties (dob: 1986-07-08, name: Renata Costa, id: 190354); Node 178667 with label Person has properties (dob: 1982-07-07, name: Rosana, id: 178667); Node 178665 with label Person has properties (dob: 1984-01-12, name: Daniela, id: 178665); Node 31 with label Person has properties (dob: 1978-03-03, name: Formiga, id: 31); Node 29 with label Person has properties (dob: 1977-04-30, name: Maycon, id: 29); Node 190358 with label Person has properties (dob: 1986-02-19, name: Marta, id: 190358); Node 190362 with label Person has properties (dob: 1985-05-15, name: Cristiane, id: 190362); Node 251049 wit

### Rule Generation

In [64]:
client = Groq(
    api_key="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",
)

In [38]:
def generator(encoded_graph, query, model, size):
    # Function to divide the text into smaller chunks
    def chunk_text(text, max_length):
        words = text.split()
        chunks = []
        current_chunk = []
        current_length = 0

        for word in words:
            if current_length + len(word) + 1 > max_length:
                chunks.append(' '.join(current_chunk))
                current_chunk = []
                current_length = 0
            current_chunk.append(word)
            current_length += len(word) + 1

        if current_chunk:
            chunks.append(' '.join(current_chunk))

        return chunks

    # Function to process each chunk
    def process_chunk(chunk):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": chunk,
                }
            ],
            model= model,
        )

        return chat_completion.choices[0].message.content

    # Split encoded_graph into chunks
    max_chunk_length = size  # Adjust this length based on your needs and model limits
    graph_chunks = chunk_text(encoded_graph, max_chunk_length)

    # Process each chunk of encoded_graph
    processed_graph_parts = [process_chunk(chunk) for chunk in graph_chunks]

    # Combine processed parts back into a single string
    processed_graph = ' '.join(processed_graph_parts)

    # Append query to the processed graph and process the final combined context
    context = processed_graph + "\n" + query
    final_result = process_chunk(context)

    return final_result

## Simple Prompt

In [139]:
prompt = """
Based on the following graph properties, generate detailed consistency rules (graph functional dependency and graph entity dependency).
Consider the structure, node information and relationships in the graph, and provide a set of rules that can be applied to maintain consistent 
and accurate data.

For each consistency rule you identify, provide a clear description of the rule and the corresponding Cypher query
"""

### LLAMA

In [45]:
rules_llama = generator(encoded_data, prompt, "llama3-70b-8192", 8000)

In [46]:
print(rules_llama)

Based on the provided graph structure and node information, I've identified a set of consistency rules that can be applied to maintain consistent and accurate data. These rules include graph functional dependencies and graph entity dependencies.

**Rule 1: Unique Person ID**
Description: Each person node should have a unique `id` property.
Cypher Query: `MATCH (p:Person) RETURN COUNT(DISTINCT p.id) AS count` should return a count of 64 (the number of nodes).

**Rule 2: Person Node Properties**
Description: Each person node should have a `name` and `dob` property.
Cypher Query: `MATCH (p:Person) WHERE NOT EXISTS(p.name) OR NOT EXISTS(p.dob) RETURN p` should return an empty set.

**Rule 3: Unique Match ID**
Description: Each match node should have a unique `id` property.
Cypher Query: `MATCH (m:Match) RETURN COUNT(DISTINCT m.id) AS count` should return a count of 34 (the number of nodes).

**Rule 4: Match Node Properties**
Description: Each match node should have a `date` and `stage` pro

### Evaluation

#### Rule 1: Unique Person ID
**Description: Each person node should have a unique `id` property** 
- **Support: Number of Person nodes that satisfy the id condition without duplicates: 2020**
  
        MATCH (p:Person)
        WITH p.id AS id, COUNT(p) AS count
        WHERE count = 1
        RETURN COUNT(*) AS support

- **Coverage: Ratio between the number of unique ids and the total number of Person nodes in the graph database: 100%**
  
        MATCH (p:Person)
        WITH p.id AS id, COUNT(p) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (p:Person)
        RETURN unique_id_count / COUNT(*) AS coverage

- **Confidence: Ratio between the number of unique ids and the total number of Person nodes with the id attribute: 100%**
  
        MATCH (p:Person)
        WITH p.id AS id, COUNT(p) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (p:Person)
        WHERE p.id IS NOT NULL
        RETURN unique_id_count / COUNT(*) AS confidence


#### Rule 2: Person Node Properties
**Description: Each person node should have a `name` and `dob` property**
- **Support: Number of Person nodes that have both name and dob attributes: 1678**

        MATCH (p:Person)
        WHERE p.name IS NOT NULL AND p.dob IS NOT NULL
        RETURN COUNT(*) AS support

- **Coverage: Ratio of the number of Person nodes that have both name and dob attributes to the total number of Person nodes: 0.8269040553907022**
  
        MATCH (p:Person)
        WHERE p.name IS NOT NULL AND p.dob IS NOT NULL
        WITH COUNT(*) AS support
        MATCH (p:Person)
        RETURN support / TOFLOAT(COUNT(*)) AS coverage

- **Confidence: Ratio of the number of Person nodes that have both name and dob attributes to the number of Person nodes that have at least one of the two attributes: 0.83**

        MATCH (p:Person)
        WHERE p.name IS NOT NULL AND p.dob IS NOT NULL
        WITH COUNT(*) AS support
        MATCH (p:Person)
        WHERE p.name IS NOT NULL OR p.dob IS NOT NULL
        RETURN support / TOFLOAT(COUNT(*)) AS confidence

#### Rule 3: Unique Match ID
**Description: Each match node should have a unique `id` property**
- **Support: Number of Match nodes that satisfy the id condition without duplicates: 284**
  
        MATCH (m:Match)
        WITH m.id AS id, COUNT(m) AS count
        WHERE count = 1
        RETURN COUNT(*) AS support

- **Coverage: Ratio between the number of unique ids and the total number of Match nodes in the graph database: 100%**
  
        MATCH (m:Matcch)
        WITH m.id AS id, COUNT(m) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (m:Match)
        RETURN unique_id_count / COUNT(*) AS coverage

- **Confidence: Ratio between the number of unique ids and the total number of Match nodes with the id attribute: 100%**
  
        MATCH (m:Match)
        WITH m.id AS id, COUNT(m) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (m:Match)
        WHERE m.id IS NOT NULL
        RETURN unique_id_count / COUNT(*) AS confidence

#### Rule 4: Match Node Properties
**Description: Each match node should have a `date` and `stage` property.**

- **Support: Number of Match nodes that have both date and stage: 284**

        MATCH (m:Match)
        WHERE m.date IS NOT NULL AND m.stage IS NOT NULL
        RETURN COUNT(*) AS support

- **Coverage: Ratio of the number of Match nodes that have both date and stage to the total number of Match nodes: 100%**

        MATCH (m:Match)
        WHERE m.date IS NOT NULL AND m.stage IS NOT NULL
        WITH COUNT(*) AS support
        MATCH (m:Match)
        RETURN support / TOFLOAT(COUNT(*)) AS coverage

- **Confidence: Ratio of the number of Match nodes that have both date and stage to the total number of Match nodes that have at least one of the two attributes date or stage: 100%**

        MATCH (m:Match)
        WHERE m.date IS NOT NULL AND m.stage IS NOT NULL
        WITH COUNT(*) AS complete_matches
        MATCH (m:Match)
        WHERE m.date IS NOT NULL OR m.stage IS NOT NULL
        RETURN complete_matches / TOFLOAT(COUNT(*)) AS confidence

#### Rule 5: Unique Squad ID
**Description: Each squad node should have a unique `id` property**

- **Support: Number of Squad nodes that satisfy the id condition without duplicates: 136**
  
        MATCH (s:Squad)
        WITH s.id AS id, COUNT(s) AS count
        WHERE count = 1
        RETURN COUNT(*) AS support

- **Coverage: Ratio between the number of unique ids and the total number of Squad nodes in the graph database: 100%**
  
        MATCH (s:Squad)
        WITH s.id AS id, COUNT(s) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (s:Squad)
        RETURN unique_id_count / COUNT(*) AS coverage

- **Confidence: Ratio between the number of unique ids and the total number of Squad nodes with the id attribute: 100%**
  
        MATCH (s:Squad)
        WITH s.id AS id, COUNT(s) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (s:Squad)
        WHERE s.id IS NOT NULL
        RETURN unique_id_count / COUNT(*) AS confidence

#### Rule 6: Squad Node Properties 
**Description: Each squad node should have an `id` property**

// Same rule 5

#### Rule 7: Unique Team ID
**Description: Each team node should have a unique `id` property**

- **Support: Number of Team nodes that satisfy the id condition without duplicates: 36**
  
        MATCH (t:Team)
        WITH t.id AS id, COUNT(t) AS count
        WHERE count = 1
        RETURN COUNT(*) AS support

- **Coverage: Ratio between the number of unique ids and the total number of Team nodes in the graph database: 100%**
  
        MATCH (t:Team)
        WITH t.id AS id, COUNT(t) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (t:Team)
        RETURN unique_id_count / COUNT(*) AS coverage

- **Confidence: Ratio between the number of unique ids and the total number of Team nodes with the id attribute: 100%**
  
        MATCH (t:Team)
        WITH t.id AS id, COUNT(t) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (t:Team)
        WHERE p.id IS NOT NULL
        RETURN unique_id_count / COUNT(*) AS confidence


#### Rule 8: Team Node Properties
**Description: Each team node should have a `name` property**

- **Support: Number of Team nodes with the name attribute: 36**

        MATCH (t:Team)
        WHERE t.name IS NOT NULL
        RETURN COUNT(*) AS support

- **Coverage: Ratio of the number of Team nodes with the name attribute to the total number of Team nodes: 100%**

        MATCH (t:Team)
        WHERE t.name IS NOT NULL
        WITH COUNT(*) AS support
        MATCH (t:Team)
        RETURN support / TOFLOAT(COUNT(*)) AS coverage

- **Confidence: Ratio of the number of Team nodes with the name attribute to the total number of Team nodes. In this case, Confidence will equal Coverage: 100%**

        MATCH (t:Team)
        WHERE t.name IS NOT NULL
        WITH COUNT(*) AS support
        MATCH (t:Team)
        RETURN support / TOFLOAT(COUNT(*)) AS confidence

#### Rule 9: Person-Match Relationship
**Description: A person node can only be connected to a match node through a `PLAYED_IN` or `SCORED_GOAL` relationship**

- **Support: number of Person nodes that have only PLAYED_IN or SCORED_GOAL relationship with Match: 1576**

        MATCH (p:Person)-[:PLAYED_IN|SCORED_GOAL]->(m:Match)
        RETURN COUNT(DISTINCT p) AS support

- **Coverage: Ratio of the number of Person nodes that have a PLAYED_IN or SCORED_GOAL relationship with Match to the total number of Person nodes that have any relationship with Match: 100%**

        MATCH (p:Person)-[:PLAYED_IN|SCORED_GOAL]->(m:Match)
        WITH COUNT(DISTINCT p) AS support
        MATCH (p:Person)-[]->(m:Match)
        RETURN support / COUNT(DISTINCT p) AS coverage

- **Confidence: 100%**

        MATCH (p:Person)-[:PLAYED_IN|SCORED_GOAL]->(m:Match)
        WITH COUNT(DISTINCT p) AS support
        MATCH (p:Person)-[:PLAYED_IN|SCORED_GOAL]->()
        RETURN support / COUNT(DISTINCT p) AS coverage

#### Rule 10: Squad-Tournament Relationship
**Description: A squad node can only be connected to a tournament node through a `FOR` relationship**

- **Support: This query counts the number of distinct squad nodes that are connected to a tournament node through a FOR relationship: 136**

        MATCH (s:Squad)-[:FOR]->(t:Tournament)
        RETURN COUNT(DISTINCT s) AS support

- **Coverage: This query first calculates the support (as above). Then, it counts the total number of squad nodes that are connected to a tournament node through any relationship. Finally, it divides the support by the total count to get the coverage: 100%**

        MATCH (s:Squad)-[:FOR]->(t:Tournament)
        WITH COUNT(DISTINCT s) AS support
        MATCH (s:Squad)-[]->(t:Tournament)
        RETURN support / COUNT(DISTINCT s) AS coverage


- **Confidence:  This query first calculates the support (as above). Then, it counts the number of squad nodes that have at least one FOR relationship. Finally, it divides the count of squad nodes that satisfy the rule body: 100%**

        MATCH (s:Squad)-[:FOR]->(t:Tournament)
        WITH COUNT(DISTINCT s) AS support
        MATCH (s:Squad)
        WHERE EXISTS((s)-[:FOR]->())
        RETURN support / (support + COUNT(DISTINCT s) - support) AS confidence


#### Rule 11: Team-Squad Relationship
**Description: A team node can only be connected to a squad node through a `NAMED` relationship**

- **Support: This query counts the number of distinct team nodes that are connected to a squad node through a NAMED relationship: 36**

        MATCH (t:Team)-[:NAMED]->(s:Squad)
        RETURN COUNT(DISTINCT t) AS support

- **Coverage: This query first calculates the support. Then, it counts the total number of team nodes that are connected to a squad node through any relationship. Finally, it divides the support by the total count to get the coverage: 100%**

        MATCH (t:Team)-[:NAMED]->(s:Squad)
        WITH COUNT(DISTINCT t) AS support
        MATCH (t:Team)-[]->(s:Squad)
        RETURN support / COUNT(DISTINCT t) AS coverage

- **Confidence: This query first calculates the support (as above). Then, it counts the number of team nodes that have at least one NAMED relationship. Finally, it divides the support by the count of team nodes that satisfy the rule body: 100%**

        MATCH (t:Team)-[:NAMED]->(s:Squad)
        WITH COUNT(DISTINCT t) AS support
        MATCH (t:Team)
        WHERE EXISTS((t)-[:NAMED]->())
        RETURN support / (COUNT(DISTINCT t)) AS confidence

#### Rule 12: Team-Tournament Relationship
**Description: A team node can only be connected to a tournament node through a `PARTICIPATED_IN` relationship**

- **Support: This query counts the number of distinct team nodes that are connected to a tournament node through a PARTICIPATED_IN relationship: 36**

        MATCH (t:Team)-[:PARTICIPATED_IN]->(to:Tournament)
        RETURN COUNT(DISTINCT t) AS support
    

- **Coverage: This query first calculates the support. Then, it counts the total number of team nodes that are connected to a tournament node through any relationship. Finally, it divides the support by the total count to get the coverage: 100%**

        MATCH (t:Team)-[:PARTICIPATED_IN]->(to:Tournament)
        WITH COUNT(DISTINCT t) AS support
        MATCH (t:Team)-[]->(to:Tournament)
        RETURN support / COUNT(DISTINCT t) AS coverage

- **Confidence: This query first calculates the support. Then, it counts the number of team nodes that have at least one PARTICIPATED_IN relationship. Finally, it divides the support by the count of team nodes that satisfy the rule body. 100%**

        MATCH (t:Team)-[:PARTICIPATED_IN]->(to:Tournament)
        WITH COUNT(DISTINCT t) AS support
        MATCH (t:Team)
        WHERE EXISTS((t)-[:PARTICIPATED_IN]->())
        RETURN support / (support + COUNT(DISTINCT t) - support) AS confidence

#### Rule 13: Match-Tournament Relationship
**Description: A match node can only be connected to a tournament node through an `IN_TOURNAMENT` relationship**

- **Support: This query counts the number of distinct match nodes that are connected to a tournament node through an IN_TOURNAMENT relationship: 284**


        MATCH (m:Match)-[:IN_TOURNAMENT]->(t:Tournament)
        RETURN COUNT(DISTINCT m) AS support

- **Coverage: This query first calculates the support. Then, it counts the total number of match nodes that are connected to a tournament node through any relationship. Finally, it divides the support by the total count to get the coverage: 100%**

        MATCH (m:Match)-[:IN_TOURNAMENT]->(t:Tournament)
        WITH COUNT(DISTINCT m) AS support
        MATCH (m:Match)-[]->(t:Tournament)
        RETURN support / COUNT(DISTINCT m) AS coverage

- **Confidence: This query first calculates the support. Then, it counts the number of match nodes that have at least one IN_TOURNAMENT relationship. Finally, it divides the support by the count of match nodes that satisfy the rule body: 100%**

        MATCH (m:Match)-[:IN_TOURNAMENT]->(t:Tournament)
        WITH COUNT(DISTINCT m) AS support
        MATCH (m:Match)
        WHERE EXISTS((m)-[:IN_TOURNAMENT]->())
        RETURN support / (support + COUNT(DISTINCT m) - support) AS confidence

### Mixtral

In [142]:
rules_mixtral = generator(encoded_data, prompt, "mixtral-8x7b-32768", 8000)

In [143]:
print(rules_mixtral)

Here are some consistency rules for the provided graph:

1. **Unique Tournament Identifier:** Each tournament should have a unique identifier. Cypher query:
```cypher
CREATE CONSTRAINT ON (t:Tournament) ASSERT t.id IS UNIQUE;
```

2. **Unique Match Identifier within a Tournament:** Each match should have a unique identifier within its respective tournament.
```cypher
CREATE CONSTRAINT ON (m:Match) ASSERT (m.tournamentId, m.id) IS UNIQUE;
```

3. **Unique Player Identifier:** Each player should have a unique identifier.
```cypher
CREATE CONSTRAINT ON (p:Person) ASSERT p.id IS UNIQUE;
```

4. **Unique Squad within a Tournament:** Each squad should have a unique identifier within its respective tournament. This rule ensures that there are no duplicate squad names in the same tournament.
```cypher
CREATE CONSTRAINT ON (s:Squad) ASSERT (s.tournamentId, s.id) IS UNIQUE;
```

5. **Existence of Squad-Team Relationship:** For any given tournament, every team should be associated with a squad.
`

#### Evaluation

#### Rule 1: Unique Tournament Identifier:
**Each tournament should have a unique identifier**

- **Support: Number of tournament nodes that satisfy the id condition without duplicates: 8**
  
            MATCH (t:Tournament)
            WITH t.id AS id, COUNT(t) AS count
            WHERE count = 1
            RETURN COUNT(*) AS support

- **Coverage: Ratio between the number of unique ids and the total number of tournament nodes in the graph database: 100%**
  
            MATCH (t:Tournament)
            WITH t.id AS id, COUNT(t) AS count
            WHERE count = 1
            WITH COUNT(*) AS unique_id_count
            MATCH (t:tournament)
            RETURN unique_id_count / COUNT(*) AS coverage

- **Confidence: Ratio between the number of unique ids and the total number of tournament nodes with the id attribute: 100%**
  
            MATCH (t:Tournament)
            WITH t.id AS id, COUNT(t) AS count
            WHERE count = 1
            WITH COUNT(*) AS unique_id_count
            MATCH (t:Tournament)
            WHERE t.id IS NOT NULL
            RETURN unique_id_count / COUNT(*) AS confidence

#### Rule 2: Unique Match Identifier within a Tournament
**Each match should have a unique identifier within its respective tournament**

- **Support: 8**
  
            MATCH (m:Match)-[:IN_TOURNAMENT]->(t:Tournament)
            WITH t, m.id AS id, COUNT(DISTINCT m) AS count
            WHERE count = 1
            RETURN COUNT(DISTINCT t) AS support

- **Coverage: 100%**

            MATCH (m:Match)-[:IN_TOURNAMENT]->(t:Tournament)
            WITH t, m.id AS match_id, COUNT(m) AS match_count
            WHERE match_count = 1
            WITH COUNT(DISTINCT t) AS valid_tournaments
            
            // Total number of tournaments
            MATCH (t:Tournament)
            WITH valid_tournaments, COUNT(t) AS total_tournaments
            
            // Compute coverage
            RETURN toFloat(valid_tournaments) / total_tournaments AS coverage
            

#### Rule 3: Unique Player Identifier
**Each player should have a unique identifier**

- **Support: Number of Person nodes that satisfy the id condition without duplicates: 2020**
  
            MATCH (p:Person)
            WITH p.id AS id, COUNT(p) AS count
            WHERE count = 1
            RETURN COUNT(*) AS support

- **Coverage: Ratio between the number of unique ids and the total number of Person nodes in the graph database: 100%**
  
            MATCH (p:Person)
            WITH p.id AS id, COUNT(p) AS count
            WHERE count = 1
            WITH COUNT(*) AS unique_id_count
            MATCH (p:Person)
            RETURN unique_id_count / COUNT(*) AS coverage

- **Confidence: Ratio between the number of unique ids and the total number of Person nodes with the id attribute: 100%**
  
            MATCH (p:Person)
            WITH p.id AS id, COUNT(p) AS count
            WHERE count = 1
            WITH COUNT(*) AS unique_id_count
            MATCH (p:Person)
            WHERE p.id IS NOT NULL
            RETURN unique_id_count / COUNT(*) AS confidence


#### Rule 4: Unique Squad within a Tournament
**Each squad should have a unique identifier within its respective tournament. This rule ensures that there are no duplicate squad names in the same tournament**

- **Support: 136**

            MATCH (s:Squad)-[r:FOR]->(t:Tournament)
            WITH t, s.properties.id AS squad_id, COUNT(DISTINCT s) AS count
            WHERE count = 1
            RETURN COUNT(DISTINCT t) AS support


- **Coverage: 100%**

            // Count of tournaments with unique squad identifiers
            MATCH (s:Squad)-[r:FOR]->(t:Tournament)
            WITH t, s.id AS squad_id, COUNT(DISTINCT s) AS squad_count
            WHERE squad_count = 1
            WITH COUNT(DISTINCT t) AS valid_tournaments
            
            // Total number of tournaments
            MATCH (t:Tournament)
            WITH valid_tournaments, COUNT(t) AS total_tournaments
            
            // Compute coverage
            RETURN toFloat(valid_tournaments) / total_tournaments AS coverage

- **Confidence: 100%**

            // Count of tournaments with unique squad identifiers
            MATCH (s:Squad)-[r:FOR]->(t:Tournament)
            WITH t, s.id AS squad_id, COUNT(DISTINCT s) AS squad_count
            WHERE squad_count = 1
            WITH COUNT(DISTINCT t) AS valid_tournaments
            
            // Total number of tournaments
            MATCH (t:Tournament)
            WITH valid_tournaments, COUNT(t) AS total_tournaments
            
            // Compute coverage
            RETURN toFloat(valid_tournaments) / total_tournaments AS confident



#### Rule 5: Existence of Squad-Team Relationship
**For any given tournament, every team should be associated with a squad**

- ***Support: 136**

            MATCH (t:Tournament)
            MATCH (team:Team)
            WHERE EXISTS {
                MATCH (team)-[:NAMED]->(s:Squad)
                WHERE EXISTS {
                    (s)-[:FOR]->(t)
                }
            }
            RETURN DISTINCT t.name AS tournament_name, team.name AS team_name


- **Support: 100%**

            MATCH (t:Tournament)
            WHERE EXISTS {
              MATCH (team:Team)
              WHERE EXISTS {
                MATCH (team)-[:NAMED]->(s:Squad)-[:FOR]->(t)
              }
            }
            WITH COUNT(DISTINCT t) AS valid_tournaments
            
            MATCH (t:Tournament)
            WITH valid_tournaments, COUNT(t) AS total_tournaments
            RETURN toFloat(valid_tournaments) / total_tournaments AS coverage

- **Confidence: 100%**

            MATCH (team:Team)-[:NAMED]->(s:Squad)-[:FOR]->(t:Tournament)
            WITH COUNT(DISTINCT team) AS teams_with_squad
            
            MATCH (team:Team)
            WITH teams_with_squad, COUNT(team) AS total_teams
            
            RETURN toFloat(teams_with_squad) / total_teams AS confidence


#### Rule 6: Existence of Team-Match Relationships 
**A team should be involved in at least one match for each tournament they participate in**


- **Support: 136**

            MATCH (t:Tournament)<-[:PARTICIPATED_IN]-(team:Team)
            MATCH (team)-[:PLAYED_IN]->(match:Match)
            WITH t, team, COUNT(match) AS matches
            WHERE matches >=1
            RETURN DISTINCT t.name AS tournament_name, team.name AS team_name


- **Coverage: 100%**

            // Tournaments where every team is involved in at least one match
            MATCH (t:Tournament)<-[:PARTICIPATED_IN]-(team:Team)
            MATCH (team)-[:PLAYED_IN]->(match:Match)
            WITH t, team, COUNT(match) AS matches
            WHERE matches >= 1
            WITH t, COLLECT(team) AS teams
            UNWIND teams AS team
            WITH t, team, (team)-[:PLAYED_IN]->(:Match) AS played_matches
            WHERE played_matches IS NOT NULL
            WITH COUNT(DISTINCT t) AS valid_tournaments
            
            // Total number of tournaments
            MATCH (t:Tournament)
            WITH valid_tournaments, COUNT(t) AS total_tournaments
            
            RETURN toFloat(valid_tournaments) / total_tournaments AS coverage

- **Confidence: 100%**

            // Teams that have played at least one match in any tournament
            MATCH (team:Team)-[:PLAYED_IN]->(match:Match)
            WITH COUNT(DISTINCT team) AS teams_with_matches
            
            // Total number of teams
            MATCH (team:Team)
            WITH teams_with_matches, COUNT(team) AS total_teams
            
            RETURN toFloat(teams_with_matches) / total_teams AS confidence


#### Rule 7: Match Score Consistency
**A match should have a score for both teams (no NULL values) if their match has a score defined**

- **Support: 284**

            MATCH (m:Match)<-[r:PLAYED_IN]-(t:Team)
            WITH m, COLLECT(r.score) AS scores
            WHERE SIZE(scores) = 2 AND ALL(score IN scores WHERE score IS NOT NULL)
            RETURN m AS match, scores

- **Coverage: 100%**

            // Count matches with scores for both teams
            MATCH (m:Match)<-[r:PLAYED_IN]-(t:Team)
            WITH m, COLLECT(r.score) AS scores
            WHERE SIZE(scores) = 2 AND ALL(score IN scores WHERE score IS NOT NULL)
            WITH COUNT(DISTINCT m) AS valid_matches_with_scores
            
            // Count all matches with scores defined
            MATCH (m:Match)<-[r:PLAYED_IN]-(t:Team)
            WITH valid_matches_with_scores, m, COLLECT(r.score) AS scores
            WHERE SIZE(scores) > 0
            WITH valid_matches_with_scores, COUNT(DISTINCT m) AS total_matches_with_scores
            
            RETURN toFloat(valid_matches_with_scores) / total_matches_with_scores AS coverage

- **Confidence: 100%**

            // Matches with scores for both teams
            MATCH (m:Match)<-[r:PLAYED_IN]-(t:Team)
            WITH m, COLLECT(r.score) AS scores
            WHERE SIZE(scores) = 2 AND ALL(score IN scores WHERE score IS NOT NULL)
            WITH COUNT(DISTINCT m) AS matches_with_both_scores
            
            // All matches with any scores defined
            MATCH (m:Match)<-[r:PLAYED_IN]-(t:Team)
            WITH matches_with_both_scores, m, COLLECT(r.score) AS scores
            WHERE SIZE(scores) > 0
            WITH matches_with_both_scores, COUNT(DISTINCT m) AS total_matches_with_scores
            
            RETURN toFloat(matches_with_both_scores) / total_matches_with_scores AS confidence



#### Rule 8:  Valid Player-Squad Relationship
**A player should be associated with a squad, and that squad should belong to the tournament for which the player has played a match**

- **Support: 7684**

            MATCH (p:Person)-[:IN_SQUAD]->(s:Squad)
            MATCH (p)-[:PLAYED_IN]->(m:Match)
            MATCH (m)-[:IN_TOURNAMENT]->(t:Tournament)
            WHERE EXISTS {
                MATCH (s)-[:FOR]->(t)
            }
            RETURN p.name AS person, s.name AS squad, t.name AS tournament


- **Coverage: 81.1%**

            MATCH (p:Person)-[:IN_SQUAD]->(s:Squad)
            MATCH (p)-[:PLAYED_IN]->(m:Match)
            MATCH (m)-[:IN_TOURNAMENT]->(t:Tournament)
            WHERE EXISTS {
                MATCH (s)-[:FOR]->(t)
            }
            WITH COUNT(DISTINCT p) AS valid_relationships
            
            // Count all player-squad relationships
            MATCH (p:Person)-[:IN_SQUAD]->(s:Squad)
            WITH valid_relationships, COUNT(DISTINCT p) AS total_relationships
            
            RETURN toFloat(valid_relationships) / total_relationships AS coverage



- **Confidence: 81.1%**

            // Count valid player-squad relationships
            MATCH (p:Person)-[:IN_SQUAD]->(s:Squad)
            MATCH (p)-[:PLAYED_IN]->(m:Match)
            MATCH (m)-[:IN_TOURNAMENT]->(t:Tournament)
            WHERE EXISTS {
                MATCH (s)-[:FOR]->(t)
            }
            WITH COUNT(DISTINCT p) AS valid_relationships
            
            // Count all player-squad relationships
            MATCH (p:Person)-[:IN_SQUAD]->(s:Squad)
            WITH valid_relationships, COUNT(DISTINCT p) AS total_relationships
            
            RETURN toFloat(valid_relationships) / total_relationships AS confidence


#### Rule 9: Valid Player-Match Goal Relationship
**A player should score a goal in a match only if they played in that match**

- **Support: 903**

            MATCH (p:Person)-[:PLAYED_IN]->(m:Match)
            MATCH (p)-[:SCORED_GOAL]->(m)
            RETURN p.name AS player, m.date AS match_date

- **Coverage: 99.2%**

            // Count valid player-goal relationships
            MATCH (p:Person)-[:PLAYED_IN]->(m:Match)
            MATCH (p)-[:SCORED_GOAL]->(m)
            WITH COUNT(DISTINCT p) AS valid_goals
            
            // Count all player-goal relationships
            MATCH (p:Person)-[:SCORED_GOAL]->(m:Match)
            WITH valid_goals, COUNT(DISTINCT p) AS total_goals
            
            RETURN toFloat(valid_goals) / total_goals AS coverage


- **Confidence: 24.5**

            // Count valid player-goal relationships
            MATCH (p:Person)-[:PLAYED_IN]->(m:Match)
            MATCH (p)-[:SCORED_GOAL]->(m)
            WITH COUNT(DISTINCT p) AS valid_goals
            
            // Count all player-match played relationships
            MATCH (p:Person)-[:PLAYED_IN]->(m:Match)
            WITH valid_goals, COUNT(DISTINCT p) AS total_played
            
            RETURN toFloat(valid_goals) / total_played AS confidence



## Few shot prompting

In [144]:
few_shot_prompt = """
    Examples of consistency Rules:
    
    1. Unique Person ID: Each Person node should have a unique id.
    2. Person Node Properties: Each Person node should have a name and dob.
    3. Ensure that no two matches have the same date, stage, and tournament. This helps avoid duplicate matches within the same tournament.
    
    Task: Generate new rules to ensure consistency and accuracy in the graph database, considering all node types and relationships. 
    For each consistency rule you identify, provide a clear description of the rule and the corresponding Cypher query for checking
    
    Requirements:
    
    Ensure data consistency across all nodes and relationships.
    Avoid rules that apply to only one type of node or relationship broader context.
"""

#### LLAMA3

In [145]:
rules_fs_llama = generator(encoded_data, few_shot_prompt, "llama3-70b-8192", 8000)

In [146]:
print(rules_fs_llama)

Based on the provided graph structure and relationships, I've identified several consistency rules to ensure data accuracy and consistency across all nodes and relationships.

**Rule 1: Unique Match ID**
Each Match node should have a unique id.

Cypher query:
```
MATCH (m1:Match{id: x}) RETURN COUNT(DISTINCT m1) == 1
```
**Rule 2: Valid Date Ranges**
Match dates should fall within a valid range (e.g., between 1900 and 2030).

Cypher query:
```
MATCH (m:Match) WHERE m.date < 1900 OR m.date > 2030 RETURN COUNT(m) == 0
```
**Rule 3: Consistent Stage Values**
Match stages should only have specific, pre-defined values (e.g., "Group Stage", "Quarter Finals", etc.).

Cypher query:
```
MATCH (m:Match) WHERE m.stage NOT IN ["Group Stage", "Quarter Finals", ...] RETURN COUNT(m) == 0
```
**Rule 4: Valid Team Names**
Team names should not be empty or null.

Cypher query:
```
MATCH (t:Team) WHERE t.name IS NULL OR t.name = "" RETURN COUNT(t) == 0
```
**Rule 5: Unique Squad Names**
Squad names shoul

### Evaluation

#### Rule 1: Unique Match ID
**Each Match node should have a unique id**

- **Support: Number of Person nodes that satisfy the id condition without duplicates: 2020**
  
        MATCH (p:Person)
        WITH p.id AS id, COUNT(p) AS count
        WHERE count = 1
        RETURN COUNT(*) AS support

- **Coverage: Ratio between the number of unique ids and the total number of Person nodes in the graph database: 100%**
  
        MATCH (p:Person)
        WITH p.id AS id, COUNT(p) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (p:Person)
        RETURN unique_id_count / COUNT(*) AS coverage

- **Confidence: Ratio between the number of unique ids and the total number of Person nodes with the id attribute: 100%**
  
        MATCH (p:Person)
        WITH p.id AS id, COUNT(p) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (p:Person)
        WHERE p.id IS NOT NULL
        RETURN unique_id_count / COUNT(*) AS confidence

#### Rule 2: Valid Date Ranges
**Match dates should fall within a valid range (e.g., between 1900 and 2030)**

- **Support: 284**

        MATCH (m:Match)
        WHERE m.date >= date("1900-01-01") AND m.date <= date("2030-12-31")
        RETURN COUNT(m) AS valid_date_count

- **Coverage: 100%**

        // Count of matches with valid dates
        MATCH (m:Match)
        WHERE m.date >= date("1900-01-01") AND m.date <= date("2030-12-31")
        WITH COUNT(m) AS valid_date_count
        
        // Count of all matches
        MATCH (m:Match)
        WITH valid_date_count, COUNT(m) AS total_matches
        
        RETURN toFloat(valid_date_count) / total_matches AS coverage


- **Confidence: 100%**

        // Count of matches with valid dates
        MATCH (m:Match)
        WHERE m.date >= date("1900-01-01") AND m.date <= date("2030-12-31")
        WITH COUNT(m) AS valid_date_count
        
        // Count of matches with dates
        MATCH (m:Match)
        WHERE m.date IS NOT NULL
        WITH valid_date_count, COUNT(m) AS total_date_count
        
        RETURN toFloat(valid_date_count) / total_date_count AS confidence


#### Rule 3: Consistent Stage Values
**Match stages should only have specific, pre-defined values (e.g., "Group Stage", "Quarter Finals", etc.)**

- **Support: 284**
  
        MATCH (m:Match)
        WHERE m.stage NOT IN [
          "Group Matches", 
          "First Stage", 
          "First stage", 
          "Round of 16", 
          "Final", 
          "Quarter Finals", 
          "Quarter-final", 
          "Match for third place", 
          "Match for 3rd place", 
          "Semifinals", 
          "Semi-final", 
          "Quarter-finals", 
          "Semi-finals"
        ]
        RETURN m.id AS match_id, m.stage AS match_stage

- **Coverage: 100%**

- **Confidence: 100%**

#### Rule 4: Valid Team Names
**Team names should not be empty or null**

- **Support: Number of Team nodes with the name attribute: 36**

        MATCH (t:Team)
        WHERE t.name IS NOT NULL
        RETURN COUNT(*) AS support

- **Coverage: Ratio of the number of Team nodes with the name attribute to the total number of Team nodes: 100%**

        MATCH (t:Team)
        WHERE t.name IS NOT NULL
        WITH COUNT(*) AS support
        MATCH (t:Team)
        RETURN support / TOFLOAT(COUNT(*)) AS coverage

- **Confidence: Ratio of the number of Team nodes with the name attribute to the total number of Team nodes. In this case, Confidence will equal Coverage: 100%**

        MATCH (t:Team)
        WHERE t.name IS NOT NULL
        WITH COUNT(*) AS support
        MATCH (t:Team)
        RETURN support / TOFLOAT(COUNT(*)) AS confidence

#### Rule 5: Unique Squad Names
**Squad names should be unique within a tournament**

- **Support: 136**

        MATCH (s:Squad)-[r:FOR]->(t:Tournament)
        WITH t, s.properties.id AS squad_id, COUNT(DISTINCT s) AS count
        WHERE count = 1
        RETURN COUNT(DISTINCT t) AS support

- **Coverage: 100%**

- **Confidence: 100%**

#### Rule 6: Person-Team Relationships**
Each Person node should be associated with at most one Team node.

- **Support: 1928**

        MATCH (p:Person)-[:REPRESENTS]->(t:Team)
        WITH p, COUNT(t) AS team_count
        WHERE team_count >= 1
        RETURN p.id AS person_id, p.name AS person_name, team_count

- **coverage: 95.3**

      MATCH (p:Person)-[r:REPRESENTS]->(t:Team)
      WITH p, COUNT(r) AS representsCount
      WHERE representsCount = 1
      WITH COUNT(p) AS validPlayers

      MATCH (p:Person)
      WITH validPlayers, COUNT(p) AS totalPlayers

      RETURN toFloat(validPlayers) / totalPlayers AS coverage

- **Confidence: 100%**

      MATCH (p:Person)
      OPTIONAL MATCH (p)-[r:REPRESENTS]->()
      WITH p, COUNT(r) AS representsCount
      WHERE representsCount <= 1
      WITH COUNT(p) AS validPlayers

      MATCH (p:Person)
      WITH validPlayers, COUNT(p) AS totalPlayers

      RETURN toFloat(validPlayers) / totalPlayers AS confidence


#### Rule 7:  Match-Team Relationships
**Each Match node should be associated with exactly two Team nodes**

- **Support: 284**

        MATCH (m:Match)<-[:PLAYED_IN]-(t:Team)
        WITH m, COUNT(t) AS team_count
        WHERE team_count = 2
        RETURN m.id AS match_id, m.date AS match_date, team_count
  
- **Coverage: 100%**

        // Count of matches with exactly two teams
        MATCH (m:Match)<-[:PLAYED_IN]-(t:Team)
        WITH m, COUNT(t) AS team_count
        WHERE team_count = 2
        WITH COUNT(DISTINCT m) AS matches_with_two_teams
        
        // Count of all matches
        MATCH (m:Match)
        RETURN toFloat(matches_with_two_teams) / COUNT(m) AS coverage

- **Confidence: 100%**
  
        // Count of matches with exactly two teams
        MATCH (m:Match)<-[:PLAYED_IN]-(t:Team)
        WITH m, COUNT(t) AS team_count
        WHERE team_count = 2
        WITH COUNT(DISTINCT m) AS matches_with_two_teams
        
        // Count of all matches
        MATCH (m:Match)
        RETURN toFloat(matches_with_two_teams) / COUNT(m) AS confidence


#### Rule 8:  Valid Goal Scorers
**A Person node can only score a goal in a Match node if they are part of the teams participating in that match**

- **Support: 756**

        MATCH (p:Person)-[sg:SCORED_GOAL]->(m:Match)
        MATCH (p)-[:REPRESENTS]->(t:Team)
        MATCH (t)-[:PLAYED_IN]->(m)
        WITH p, m, COUNT(t) AS valid_team_count
        WHERE valid_team_count > 0
        RETURN p.id AS person_id, p.name AS person_name, m.id AS match_id, m.date AS match_date

- **Coverage: 42.5%**

      // Total number of SCORED_GOAL relationships
      MATCH (p:Person)-[sg:SCORED_GOAL]->(m:Match)
      WITH COUNT(sg) AS totalGoals

      // Number of unique goals (i.e., no duplicate goals by the same player in the same minute of the same game)
      MATCH (p:Person)-[sg:SCORED_GOAL]->(m:Match)
      WITH totalGoals, p, m, sg.minute AS goal_minute, COUNT(sg) AS goal_count
      WHERE goal_count = 1
      WITH COUNT(DISTINCT p) AS validGoals, totalGoals

      // Calculate coverage
      RETURN toFloat(validGoals) / totalGoals AS coverage


- **Confidence: 100%**

      MATCH (p:Person)-[sg:SCORED_GOAL]->(m:Match)
      WITH COUNT(sg) AS totalGoals

      // Number of valid SCORED_GOAL relationships (i.e., unique goals in the same minute of the same game)
      MATCH (p:Person)-[sg:SCORED_GOAL]->(m:Match)
      WITH totalGoals, sg, p, m, sg.minute AS goal_minute, COUNT(sg) AS goal_count
      WHERE goal_count = 1
      WITH COUNT(sg) AS validGoals, totalGoals

      // Calculate confidence
      RETURN toFloat(validGoals) / totalGoals AS confidence


#### Mixtral

In [147]:
rules_fs_mixtral = generator(encoded_data, few_shot_prompt, "mixtral-8x7b-32768", 8000)

In [149]:
print(rules_fs_mixtral)

Sure, here are some consistency rules that can help ensure the accuracy and consistency of the graph database:

1. Unique Match ID: Each Match node should have a unique id.

   Cypher Query:
   ```sql
   MATCH (m:Match)
   RETURN DISTINCT m.id AS match_id
   ```

2. Match Date Uniqueness: Each Match node should have a unique date and stage within a specific tournament.

   Cypher Query:
   ```sql
   MATCH (m:Match)-[:PLAYED_IN]->(t:Team)-[:PARTICIPATED_IN]->(tournament:Tournament)
   WITH tournament, m.date AS match_date, m.stage AS match_stage, m.id AS match_id
   RETURN tournament.name, match_date, match_stage, COUNT(DISTINCT match_id) AS count
   ```

3. Unique Team Name: Each Team node should have a unique name within a specific tournament.

   Cypher Query:
   ```sql
   MATCH (t:Team)-[:PARTICIPATED_IN]->(tournament:Tournament)
   WITH tournament, t.name AS team_name, t.id AS team_id
   RETURN tournament.name, team_name, COUNT(DISTINCT team_id) AS count
   ```

4. Unique Tournamen

### Evaluation

#### Rule 1: Unique Match ID
**Each Match node should have a unique id**

- **Support: Number of Person nodes that satisfy the id condition without duplicates: 2020**
  
        MATCH (p:Person)
        WITH p.id AS id, COUNT(p) AS count
        WHERE count = 1
        RETURN COUNT(*) AS support

- **Coverage: Ratio between the number of unique ids and the total number of Person nodes in the graph database: 100%**
  
        MATCH (p:Person)
        WITH p.id AS id, COUNT(p) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (p:Person)
        RETURN unique_id_count / COUNT(*) AS coverage

- **Confidence: Ratio between the number of unique ids and the total number of Person nodes with the id attribute: 100%**
      
        MATCH (p:Person)
        WITH p.id AS id, COUNT(p) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (p:Person)
        WHERE p.id IS NOT NULL
        RETURN unique_id_count / COUNT(*) AS confidence

#### Rule 2: Match Date Uniqueness
**Each Match node should have a unique date and stage within a specific tournament**

-**Support: 39**

        MATCH (m:Match)-[:IN_TOURNAMENT]->(t:Tournament)
        WITH t, m.date AS match_date, m.stage AS match_stage, COUNT(*) AS count
        WHERE count = 1
        RETURN t.name AS tournament_name, match_date, match_stage, count

- **Coverage: 100%**

        // Count of matches with unique date and stage
        MATCH (m:Match)-[:IN_TOURNAMENT]->(t:Tournament)
        WITH t,m, m.date AS match_date, m.stage AS match_stage, COUNT(*) AS count
        WHERE count = 1
        WITH COUNT(DISTINCT m) AS unique_matches
        
        // Count of all matches
        MATCH (m:Match)-[:IN_TOURNAMENT]->(t:Tournament)
        RETURN toFloat(unique_matches) / COUNT(DISTINCT m) AS coverage

- **Confidence: 100%**

        // Count of matches with unique date and stage
        MATCH (m:Match)-[:IN_TOURNAMENT]->(t:Tournament)
        WITH t, m, m.date AS match_date, m.stage AS match_stage, COUNT(*) AS count
        WHERE count = 1
        WITH COUNT(DISTINCT m) AS unique_matches
        
        // Count of matches with valid date and stage information
        MATCH (m:Match)-[:IN_TOURNAMENT]->(t:Tournament)
        WHERE m.date IS NOT NULL AND m.stage IS NOT NULL
        RETURN toFloat(unique_matches) / COUNT(DISTINCT m) AS confidence



#### Rule 3: Unique Team Name
**Each Team node should have a unique name within a specific tournament**

- **Support: Number of Team nodes with the name attribute: 36**

        MATCH (t:Team)
        WHERE t.name IS NOT NULL
        RETURN COUNT(*) AS support

- **Coverage: Ratio of the number of Team nodes with the name attribute to the total number of Team nodes: 100%**

        MATCH (t:Team)
        WHERE t.name IS NOT NULL
        WITH COUNT(*) AS support
        MATCH (t:Team)
        RETURN support / TOFLOAT(COUNT(*)) AS coverage

- **Confidence: Ratio of the number of Team nodes with the name attribute to the total number of Team nodes. In this case, Confidence will equal Coverage: 100%**

        MATCH (t:Team)
        WHERE t.name IS NOT NULL
        WITH COUNT(*) AS support
        MATCH (t:Team)
        RETURN support / TOFLOAT(COUNT(*)) AS confidence

#### Rule 4: Unique Tournament Name
**Each Tournament node should have a unique name**

- **Support: Number of tournament nodes that satisfy the id condition without duplicates: 8**
  
        MATCH (t:Tournament)
        WITH t.id AS id, COUNT(t) AS count
        WHERE count = 1
        RETURN COUNT(*) AS support

- **Coverage: Ratio between the number of unique ids and the total number of tournament nodes in the graph database: 100%**
  
        MATCH (t:Tournament)
        WITH t.id AS id, COUNT(t) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (t:tournament)
        RETURN unique_id_count / COUNT(*) AS coverage

- **Confidence: Ratio between the number of unique ids and the total number of tournament nodes with the id attribute: 100%**
  
        MATCH (t:Tournament)
        WITH t.id AS id, COUNT(t) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (t:Tournament)
        WHERE t.id IS NOT NULL
        RETURN unique_id_count / COUNT(*) AS confidence

#### Rule 5: Unique Squad 
**Each Squad node should have a unique id**

- **Support: Number of Squad nodes that satisfy the id condition without duplicates: 136**
  
        MATCH (s:Squad)
        WITH s.id AS id, COUNT(s) AS count
        WHERE count = 1
        RETURN COUNT(*) AS support

- **Coverage: Ratio between the number of unique ids and the total number of Squad nodes in the graph database: 100%**
  
        MATCH (s:Squad)
        WITH s.id AS id, COUNT(s) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (s:Squad)
        RETURN unique_id_count / COUNT(*) AS coverage

- **Confidence: Ratio between the number of unique ids and the total number of Squad nodes with the id attribute: 100%**
  
        MATCH (s:Squad)
        WITH s.id AS id, COUNT(s) AS count
        WHERE count = 1
        WITH COUNT(*) AS unique_id_count
        MATCH (s:Squad)
        WHERE s.id IS NOT NULL
        RETURN unique_id_count / COUNT(*) AS confidence

#### Rule 6: Person-Squad Relationship Consistency
**Ensure that for each Person-Squad relationship, there is one and only one Squad that represents the Person**

- **Support: 1268**

        MATCH (p:Person)-[:IN_SQUAD]->(s:Squad)
        WITH p, COUNT(s) AS squad_count
        WHERE squad_count > 1
        RETURN p.id AS person_id, p.name AS person_name, squad_count

- **Coverage: 100%**
  
        // Calculate the total number of IN_SQUAD relationships
        MATCH (p:Person)-[r:IN_SQUAD]->(s:Squad)
        WITH COUNT(r) AS totalRelationships

        // Calculate the number of valid IN_SQUAD relationships
        MATCH (p:Person)-[r:IN_SQUAD]->(s:Squad)
        WITH r, totalRelationships, p, r.startDate AS start_date, r.endDate AS end_date, COUNT(s) AS squad_count
        WHERE squad_count = 1
        WITH COUNT(r) AS validRelationships, totalRelationships

        // Calculate coverage
        RETURN toFloat(validRelationships) / totalRelationships AS 
        coverage

- **Confidence: 66%**

        // Calculate the total number of COACH_FOR relationships
        MATCH (p:Person)-[r:IN_SQUAD]->(s:Squad)
        WITH COUNT(r) AS totalRelationships

        // Calculate the number of valid COACH_FOR relationships
        MATCH (p:Person)-[r:IN_SQUAD]->(s:Squad)
        WITH totalRelationships, p, r.startDate AS start_date, r.endDate AS end_date, COUNT(s) AS team_count
        WHERE team_count = 1
        WITH COUNT(DISTINCT p) AS validNodes, totalRelationships

        // Calculate total nodes with COACH_FOR relationships
        MATCH (p:Person)-[r:IN_SQUAD]->()
        WITH validNodes, totalRelationships, COUNT(DISTINCT p) AS totalNodesWithRelationships

        // Calculate confidence
        RETURN toFloat(validNodes) / totalNodesWithRelationships AS confidence


#### Rule 7: Match-Team-Tournament Relationship Consistency
**Ensure that for each Match-Team Relationship in a specific tournament, the Tournament node and Team node combinations are correct**

- **Support: 568**

        MATCH (t:Tournament)<-[:IN_TOURNAMENT]-(m:Match)<-[:PLAYED_IN]-(team:Team)
        WITH t, m, team, COUNT(*) AS count
        WHERE count= 1
        RETURN t.id AS tournament_id, t.name AS tournament_name, 
               m.id AS match_id, m.date AS match_date, 
               team.id AS team_id, team.name AS team_name, 
               count AS relationship_count

- **Coverage: 100%**

- **Confidence: 100%**

#### Rule 8: SCORED_GOAL Relationship Consistency
**Ensure that no two SCORED_GOAL relationships between a Person and Match should have the same minute property**

- **Support: 907**

        MATCH (p:Person)-[sg:SCORED_GOAL]->(m:Match)
        WITH p, m, sg.minute AS goal_minute, COUNT(sg) AS count
        WHERE count = 1
        RETURN p.id AS person_id, p.name AS person_name, 
               m.id AS match_id, m.date AS match_date, 
               goal_minute AS scored_minute, count AS number_of_goals

- **Coverage: 42.5%**

      // Total number of SCORED_GOAL relationships
      MATCH (p:Person)-[sg:SCORED_GOAL]->(m:Match)
      WITH COUNT(sg) AS totalGoals

      // Number of unique goals (i.e., no duplicate goals by the same player in the same minute of the same game)
      MATCH (p:Person)-[sg:SCORED_GOAL]->(m:Match)
      WITH totalGoals, p, m, sg.minute AS goal_minute, COUNT(sg) AS goal_count
      WHERE goal_count = 1
      WITH COUNT(DISTINCT p) AS validGoals, totalGoals

      // Calculate coverage
      RETURN toFloat(validGoals) / totalGoals AS coverage


- **Confidence: 100%**

      MATCH (p:Person)-[sg:SCORED_GOAL]->(m:Match)
      WITH COUNT(sg) AS totalGoals

      // Number of valid SCORED_GOAL relationships (i.e., unique goals in the same minute of the same game)
      MATCH (p:Person)-[sg:SCORED_GOAL]->(m:Match)
      WITH totalGoals, sg, p, m, sg.minute AS goal_minute, COUNT(sg) AS goal_count
      WHERE goal_count = 1
      WITH COUNT(sg) AS validGoals, totalGoals

      // Calculate confidence
      RETURN toFloat(validGoals) / totalGoals AS confidence

## Discuss about prompt and llms

## Repair 
### A dirty graph that need to repair

In [129]:
# read JSON file
with open('sub_graph_1.json', 'r', encoding='utf-8-sig') as file:
    data = json.load(file)


In [130]:
from pyvis.network import Network

# Khởi tạo một đồ thị bằng networkx
G = nx.Graph()

# Thêm các nút và cạnh từ dữ liệu JSON
for entry in data:
    person = entry["p"]
    person_label = f"{person['properties']['name']} ({', '.join(person['labels'])}, ID: {person['elementId']})"
    G.add_node(person["elementId"], label=person_label, name=person["properties"]["name"])
    
    for relation, related_node in zip(entry["relationships"], entry["relatedNodes"]):
        related_node_name = related_node["properties"].get("name", f"Node_{related_node['elementId']}")
        related_node_label = f"{related_node_name} ({', '.join(related_node['labels'])}, ID: {related_node['elementId']})"
        G.add_node(related_node["elementId"], label=related_node_label, name=related_node_name)
        
        # Thêm cạnh với tên mối quan hệ
        G.add_edge(person["elementId"], related_node["elementId"], label=relation["type"], title=relation["type"])

# Tạo visualizer
net = Network(notebook=True)

# Thêm các nút và cạnh vào mạng lưới pyvis
for node in G.nodes(data=True):
    net.add_node(node[0], label=node[1]["label"], title=node[1]["label"])

for edge in G.edges(data=True):
    net.add_edge(edge[0], edge[1], title=edge[2]["label"], label=edge[2]["label"])

# Hiển thị đồ thị
net.show("graph.html")


graph.html


#### Encode graph: 

In [133]:
# Hàm encode các nút (nodes)
def encode_node(node):
    label = node["labels"][0]
    properties = ", ".join([f"{key}: {value}" for key, value in node["properties"].items()])
    node_id = node["properties"].get("id", "unknown_id")  # Sử dụng "unknown_id" nếu "id" không tồn tại
    return f"Node {node_id} with label {label} has properties ({properties})"

# Hàm encode các mối quan hệ (relationships)
def encode_relationship(node_start, node_end, relationship):
    start_id = node_start["properties"].get("id", "unknown_id")
    end_id = node_end["properties"].get("id", "unknown_id")
    rel_type = relationship["type"]
    properties = ", ".join([f"{key}: {value}" for key, value in relationship["properties"].items()])
    if properties:
        return f"Node {start_id} connect to node {end_id} with Relationship Type {rel_type} and Properties {{{properties}}}"
    else:
        return f"Node {start_id} connect to node {end_id} with Relationship Type {rel_type}"

# Tạo danh sách chứa kết quả
encoded_graph = []

# Encode các nodes và relationships từ dữ liệu JSON
for entry in data:
    person = entry["p"]
    encoded_graph.append(encode_node(person))
    
    for relation, related_node in zip(entry["relationships"], entry["relatedNodes"]):
        encoded_graph.append(encode_node(related_node))
        encoded_graph.append(encode_relationship(person, related_node, relation))

# Kết quả cuối cùng
final_result = "; ".join(encoded_graph)
print(final_result)

Node 39 with label Person has properties (dob: 1977-09-14, name: Andreia, id: 39); Node 1882881 with label Club has properties (name: Brazil, id: 1882881); Node 39 connect to node 1882881 with Relationship Type REPRESENTED_BY; Node unknown_id with label Match has properties (date: 2007-09-23, stage: Final); Node 39 connect to node unknown_id with Relationship Type PLAYED_FOR and Properties {type: Subbed On}; Node 56328 with label Game has properties (date: 2007-09-15, stage: First stage, id: 56328); Node 39 connect to node 56328 with Relationship Type PLAYED_IN and Properties {type: Started}; Node 201416 with label Player has properties (dob: 1982-11-01, name: Elaine, id: 201416); Node 1882881 with label Team has properties (name: Brazil, id: 1882881); Node 201416 connect to node 1882881 with Relationship Type REPRESENTS; Node 56326 with label Match has properties (date: 2007-09-21, stage: First stage, id: 56326); Node 201416 connect to node 56326 with Relationship Type WON and Propert

### Generate rules from dirty graph

In [134]:
rules_dirty_llama = generator(final_result, prompt, "llama3-70b-8192", 8000)

In [125]:
print(rules_dirty_llama)

Based on the provided graph database, I've identified the following consistency rules:

**Graph Functional Dependency Rules:**

1. **Each `Person` node must represent only one `Team` node**:
Description: A person can only represent one team.
Cypher query:
```cypher
MATCH (p:Person)-[:REPRESENTS]->(t:Team)
WITH p, COUNT(DISTINCT t) AS team_count
WHERE team_count > 1
RETURN p, team_count
```
This query checks if a person node has more than one `REPRESENTS` relationship with different teams. If the result is non-empty, it indicates a consistency error.

2. **Each `Match` node must have a unique `date` property**:
Description: No two matches can have the same date.
Cypher query:
```cypher
MATCH (m1:Match), (m2:Match)
WHERE m1.date = m2.date AND m1 <> m2
RETURN m1, m2
```
This query checks if there are multiple `Match` nodes with the same `date` property. If the result is non-empty, it indicates a consistency error.

**Graph Entity Dependency Rules:**

1. **A `Person` node must exist before

### Define the rules

In [126]:
prompt_refine = """Let's refind consistency rules from dirty graph data. 
    Utilize information from clean graph data to improve and adjust the rules generated from dirty graph data. 
    The goal is to ensure that the new rules accurately reflect the structure and standards of the data,
    address errors and inconsistencies in the dirty graph data, while still adhering to the rules from the clean graph data.  
    Finally, generate a finally rules.
    This is generated rules from clean graph """ +  rules_llama + ". And this is generated rules from dirty graph: "

refined_rules = generator(prompt_refine, rules_dirty_llama, "llama3-70b-8192", 8000)

In [128]:
for rule in refined_rules.split("\n"): 
    print(rule)

Excellent approach! You've outlined a clear and comprehensive step-by-step process for refining consistency rules from dirty graph data by integrating them with clean graph data rules. Your example rules demonstrate how to identify and address errors and inconsistencies in the graph data, ensuring data integrity and accuracy.

The four-step approach you provided is well-structured and easy to follow:

1. Analyzing dirty graph data to identify inconsistencies and errors.
2. Refining rules from dirty graph data to address these issues.
3. Integrating the refined rules with clean graph data rules, ensuring no conflicts.
4. Generating a final set of consistency rules that reflect the correct structure and standards of the data.

The example rules you provided are well-crafted and cover various aspects of graph data consistency, including:

* Unique node IDs
* Required node properties (e.g., `name` and `dob` for person nodes)
* Functional dependencies (e.g., each person node represents only

### Repair using defined rules